<a href="https://colab.research.google.com/github/DaniPalma3000/Calendarizador/blob/master/Examen1Practico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Información Personal

Nombre: Daniel Palma

Carné: 21000429

## Preparations

In [85]:
# ----- Libraries ----- #

# This is the main Library that allows us to work with Neural Networks
import tensorflow as tf

# For graph plotting
import matplotlib.pyplot as plt
from tensorflow.math import confusion_matrix

# For dataset manipulation
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

# For visualizing more complex graphs
import seaborn as sns

# Miscellaneous
import os
import datetime

# Global constant for training acceleration
AUTOTUNE = tf.data.AUTOTUNE

# 1) Dataset Preparations

In [86]:
# Obtenemos los datos de ambos datasets

# Datos de entrenamiento separados por comas
train_raw_dataset = pd.read_csv("laptop_dataset.csv", sep=",")

train_raw_dataset

# Datos de evaluación separados por comas
evaluation_raw_dataset = pd.read_csv("evaluacion_laptop_dataset.csv", sep=",")

evaluation_raw_dataset

,category,os,gpu,cpu,cpu_frequency,ram,storage,weight,screen_size,screen_type,price
0,Netbook,Windows,Intel,Intel Pentium i7,2.7,8,256,1.36,30.480,IPS Panel,2096
1,Ultrabook,Windows,Intel,Intel Pentium i7,2.7,8,256,1.36,35.560,IPS Panel,2223
2,Notebook,Windows,Intel,Intel Pentium i5,2.5,8,256,1.25,35.560,IPS Panel,1950
3,Workstation,Linux,AMD,Intel Pentium i3,2.0,4,128,2.20,39.624,Full HD,616
4,Ultrabook,Windows,NVidia,Intel Pentium i7,2.8,8,256,1.96,35.560,IPS Panel,2417
5,Notebook,Windows,Intel,Intel Pentium i7,2.7,8,256,1.58,35.560,IPS Panel,1880
6,Notebook,Windows,Intel,Intel Pentium i5,2.5,8,256,1.56,35.560,IPS Panel,1146
7,Notebook,Windows,Intel,Intel Pentium i5,2.3,8,256,1.95,35.560,Full HD,1499
8,Gaming,Windows,NVidia,Intel Pentium i7,2.8,16,256,2.50,38.100,IPS Panel,1371
9,Notebook,Windows,NVidia,Intel Pentium i5,1.6,8,256,2.10,39.624,Full HD,1117


In [87]:
# Estadísticas del dataset de entrenamiento

train_raw_dataset.describe()

,cpu_frequency,ram,storage,weight,screen_size,price
count,218.000000,218.000000,218.000000,218.000000,218.000000,218.000000
mean,2.357339,7.844037,244.844037,1.871239,37.351982,1451.669725
std,0.410985,2.442611,36.187218,0.499857,2.984688,575.387028
min,1.200000,4.000000,128.000000,0.810000,30.480000,527.000000
25%,2.000000,8.000000,256.000000,1.480000,35.560000,1059.250000
50%,2.500000,8.000000,256.000000,1.875000,38.100000,1326.000000
75%,2.700000,8.000000,256.000000,2.200000,39.624000,1737.000000
max,2.900000,16.000000,256.000000,3.600000,43.942000,3810.000000


In [88]:
# Tipos de las columnas del dataset de entrenamiento

train_raw_dataset.dtypes

,0
category,object
os,object
gpu,object
cpu,object
cpu_frequency,float64
ram,int64
storage,int64
weight,float64
screen_size,float64
screen_type,object


In [89]:
# Dimensiones del dataset de prueba

train_raw_dataset.shape

(218, 11)

# 2) Pre-processing

## Missing Values

In [90]:

# Comenzamos haciendo una copia del dataset
new_dataset = train_raw_dataset.copy()

# Se verifican los valores nulos
print(new_dataset.isna().sum())

# Como para este caso no tenemos valores nulos,
# no realizamos ninguna otra acción
new_dataset.head()

category         0
os               0
gpu              0
cpu              0
cpu_frequency    0
ram              0
storage          0
weight           0
screen_size      0
screen_type      0
price            0
dtype: int64


,category,os,gpu,cpu,cpu_frequency,ram,storage,weight,screen_size,screen_type,price
0,Gaming,Windows,NVidia,Intel Pentium i7,2.8,8,256,2.50,39.624,Full HD,1269
1,Notebook,Linux,Intel,Intel Pentium i3,2.0,4,256,2.00,39.624,Full HD,527
2,Notebook,Linux,AMD,Intel Pentium i7,2.7,8,256,2.33,39.624,Full HD,989
3,Notebook,Windows,Intel,Intel Pentium i5,1.6,8,256,2.05,39.624,Full HD,939
4,Ultrabook,Windows,Intel,Intel Pentium i7,2.8,8,256,1.36,35.560,Full HD,2255


## Encoding

In [91]:
# Datos únicos para catgory

print(len(new_dataset["category"].unique()), new_dataset["category"].unique())

5 ['Gaming' 'Notebook' 'Ultrabook' 'Workstation' 'Netbook']


In [92]:
# Datos únicos para el os

print(len(new_dataset["os"].unique()), new_dataset["os"].unique())

2 ['Windows' 'Linux']


In [93]:
# Datos únicos para la gpu

print(len(new_dataset["gpu"].unique()), new_dataset["gpu"].unique())

3 ['NVidia' 'Intel' 'AMD']


In [94]:
# Datos únicos para el cpu

print(len(new_dataset["cpu"].unique()), new_dataset["cpu"].unique())

3 ['Intel Pentium i7' 'Intel Pentium i3' 'Intel Pentium i5']


In [95]:
# Datos únicos para el screen_type

print(len(new_dataset["screen_type"].unique()), new_dataset["screen_type"].unique())

2 ['Full HD' 'IPS Panel']


In [96]:
# Función para clasificar category

def map_category(category):
  if category == "Gaming":
    return 0
  elif category == "Notebook":
    return 1
  elif category == "Ultrabook":
    return 2
  elif category == "Workstation":
    return 3
  elif category == "Netbook":
    return 4
  else:
    return None

In [97]:
# Función para clasificar os

def map_os(os):
  if os == "Windows":
    return 0
  elif os == "Linux":
    return 1
  else:
    return None

In [98]:
# Función para clasificar gpu

def map_gpu(gpu):
  if gpu == "NVidia":
    return 0
  elif gpu == "Intel":
    return 1
  elif gpu == "AMD":
    return 2
  else:
    return None

In [99]:
# Función para clasificar cpu

def map_cpu(cpu):
  if cpu == "Intel Pentium i7":
    return 7
  elif cpu == "Intel Pentium i3":
    return 3
  elif cpu == "Intel Pentium i5":
    return 5
  else:
    return None

In [100]:
# Función para clasificar screen_type

def map_screen_type(screen_type):
  if screen_type == "Full HD":
    return 0
  elif screen_type == "IPS Panel":
    return 1
  else:
    return None

In [101]:
# Aplicamos los diferentes mapeos a las columnas necesarias

new_dataset["category"] = new_dataset["category"].apply(map_category)
new_dataset["os"] = new_dataset["os"].apply(map_os)
new_dataset["gpu"] = new_dataset["gpu"].apply(map_gpu)
new_dataset["cpu"] = new_dataset["cpu"].apply(map_cpu)
new_dataset["screen_type"] = new_dataset["screen_type"].apply(map_screen_type)

new_dataset


,category,os,gpu,cpu,cpu_frequency,ram,storage,weight,screen_size,screen_type,price
0,0,0,0,7,2.8,8,256,2.50,39.624,0,1269
1,1,1,1,3,2.0,4,256,2.00,39.624,0,527
2,1,1,2,7,2.7,8,256,2.33,39.624,0,989
3,1,0,1,5,1.6,8,256,2.05,39.624,0,939
4,2,0,1,7,2.8,8,256,1.36,35.560,0,2255
...,...,...,...,...,...,...,...,...,...,...,...
213,1,0,1,7,2.9,8,256,2.31,39.624,0,1813
214,0,0,0,5,2.5,8,256,2.65,39.624,0,1518
215,1,0,1,5,2.5,8,256,1.05,33.020,1,2124
216,1,1,0,7,2.8,8,256,2.50,39.624,0,1244


## Normalization

In [102]:
# Comenzamos a definir nuestra normalizacion para los valores, utilizaremos una funcion de min_max
# Obtendremos el valor máximo y mínimos de la columna enviada,
# y luego procedemos a operar para tener los datos normalizados

def normalize_min_max(columna):
  max_value = np.max(columna)
  min_value = np.min(columna)
  return (columna - min_value)/(max_value - min_value)

In [103]:
# Colocamos las columnas que vamos a normalizar, van a ser todas menos la del precio

columnas_normalizar = ["os", "gpu", "cpu", "cpu_frequency", "ram", "storage", "weight", "screen_size", "screen_type"]

columnas_normalizar

['os',
 'gpu',
 'cpu',
 'cpu_frequency',
 'ram',
 'storage',
 'weight',
 'screen_size',
 'screen_type']

In [104]:
dataset_normalizado = new_dataset.copy()

dataset_normalizado[columnas_normalizar] = dataset_normalizado[columnas_normalizar].apply(normalize_min_max)

dataset_normalizado

,category,os,gpu,cpu,cpu_frequency,ram,storage,weight,screen_size,screen_type,price
0,0,0.0,0.0,1.0,0.941176,0.333333,1.0,0.605735,0.679245,0.0,1269
1,1,1.0,0.5,0.0,0.470588,0.000000,1.0,0.426523,0.679245,0.0,527
2,1,1.0,1.0,1.0,0.882353,0.333333,1.0,0.544803,0.679245,0.0,989
3,1,0.0,0.5,0.5,0.235294,0.333333,1.0,0.444444,0.679245,0.0,939
4,2,0.0,0.5,1.0,0.941176,0.333333,1.0,0.197133,0.377358,0.0,2255
...,...,...,...,...,...,...,...,...,...,...,...
213,1,0.0,0.5,1.0,1.000000,0.333333,1.0,0.537634,0.679245,0.0,1813
214,0,0.0,0.0,0.5,0.764706,0.333333,1.0,0.659498,0.679245,0.0,1518
215,1,0.0,0.5,0.5,0.764706,0.333333,1.0,0.086022,0.188679,1.0,2124
216,1,1.0,0.0,1.0,0.941176,0.333333,1.0,0.605735,0.679245,0.0,1244


In [105]:
# Procedemos a normalizar también nuestros datos de evaluación aplicando los pasos previamente
# realizados con el dataset de entrenamiento

# Pasamos a datos numéricos

evaluation_dataset_normalizado = evaluation_raw_dataset.copy()

evaluation_dataset_normalizado["category"] = evaluation_dataset_normalizado["category"].apply(map_category)

evaluation_dataset_normalizado["os"] = evaluation_dataset_normalizado["os"].apply(map_os)

evaluation_dataset_normalizado["gpu"] = evaluation_dataset_normalizado["gpu"].apply(map_gpu)

evaluation_dataset_normalizado["cpu"] = evaluation_dataset_normalizado["cpu"].apply(map_cpu)

evaluation_dataset_normalizado["screen_type"] = evaluation_dataset_normalizado["screen_type"].apply(map_screen_type)

evaluation_dataset_normalizado

# Aplicamos ya la normalizacion de los datos

evaluation_dataset_normalizado[columnas_normalizar] = evaluation_dataset_normalizado[columnas_normalizar].apply(normalize_min_max)

evaluation_dataset_normalizado




,category,os,gpu,cpu,cpu_frequency,ram,storage,weight,screen_size,screen_type,price
0,4,0.0,0.5,1.0,0.916667,0.333333,1.0,0.088,0.000000,1.0,2096
1,2,0.0,0.5,1.0,0.916667,0.333333,1.0,0.088,0.555556,1.0,2223
2,1,0.0,0.5,0.5,0.750000,0.333333,1.0,0.000,0.555556,1.0,1950
3,3,1.0,1.0,0.0,0.333333,0.000000,0.0,0.760,1.000000,0.0,616
4,2,0.0,0.0,1.0,1.000000,0.333333,1.0,0.568,0.555556,1.0,2417
5,1,0.0,0.5,1.0,0.916667,0.333333,1.0,0.264,0.555556,1.0,1880
6,1,0.0,0.5,0.5,0.750000,0.333333,1.0,0.248,0.555556,1.0,1146
7,1,0.0,0.5,0.5,0.583333,0.333333,1.0,0.560,0.555556,0.0,1499
8,0,0.0,0.0,1.0,1.000000,1.000000,1.0,1.000,0.833333,1.0,1371
9,1,0.0,0.0,0.5,0.000000,0.333333,1.0,0.680,1.000000,0.0,1117


# 3) AI Model


## Callbacks

In [106]:
# Aplicamos el earlystopping para evitar caer en un ovefitting

earlystopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=5,)

## Modelo

In [107]:
# Vamos a separas nuestra variable dependiente para ambos casos, para el training y la evaluación
X_train = dataset_normalizado.drop('price', axis=1)
y_train = dataset_normalizado['price']

# Separate features (X) and target (y) for the evaluation dataset
X_test = evaluation_dataset_normalizado.drop('price', axis=1)
y_test = evaluation_dataset_normalizado['price']

# Separamos data para validacion y entrenamiento
# 80% para el entrenamiento y 20% para la validacion
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Configuramos el inpput shape a 10 por la cantidad de parámetros y
normal_model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(10,)),
  tf.keras.layers.Dense(units=32, activation='relu'),
  tf.keras.layers.Dense(units=32, activation='relu'),
  tf.keras.layers.Dense(units=32, activation='relu'),
  tf.keras.layers.Dense(units=1, activation=None)
  ])

# Compilamos el modelo y utilizamos en este caso la función loss de MeanSquaredError
normal_model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer='adam', metrics=['mean_absolute_error'])

# Entrenamos el modelo
normal_model.fit(X_train_split,
          y_train_split,
          epochs=500,
          batch_size = 128 ,
          validation_data=(X_val, y_val),
          callbacks=[earlystopping_callback]
        )

Epoch 1/500


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 370ms/step - loss: 2481156.2500 - mean_absolute_error: 1453.5338 - val_loss: 2404591.5000 - val_mean_absolute_error: 1472.5063
Epoch 2/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - loss: 2461527.5000 - mean_absolute_error: 1444.1373 - val_loss: 2404441.7500 - val_mean_absolute_error: 1472.4570
Epoch 3/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 2431531.5000 - mean_absolute_error: 1448.0294 - val_loss: 2404276.2500 - val_mean_absolute_error: 1472.4025
Epoch 4/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 2450663.7500 - mean_absolute_error: 1442.7566 - val_loss: 2404093.5000 - val_mean_absolute_error: 1472.3425
Epoch 5/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 2465398.7500 - mean_absolute_error: 1450.2339 - val_loss: 2403891.7500 - val_mean_absolute_error: 1472.2762
Epoch 6/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - loss: 2481601.7500 - mean_absolute_error: 1457.0444 - val_loss: 2403666.5000 - val_mean_absolute_error: 1472.2019
Epoch 7/500
2/

In [111]:
# Realizamos la predicción para el modelo
y_pred = normal_model.predict(X_test)

# Obtenemos el valor de RMSE
mse = tf.keras.metrics.MeanSquaredError()
mse.update_state(y_test, y_pred)
rmse = np.sqrt(mse.result().numpy())

print(f"Root Mean Squared Error (RMSE) on evaluation data: {rmse:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Root Mean Squared Error (RMSE) on evaluation data: 422.70
